Note to maintainers: "normal" python types cannot be used in non-generative functions; typical error message: 

    issubclass() arg 1 must be a class
    
To fix, add (generative=True) to @qfunc annotation.

In [77]:
from classiq import *
from classiq.qmod.symbolic import floor
import numpy as np
from typing import Union

#nth_to_last(0, list) == list[-1]
#def nth_to_last(n:int, list): 
#    return list[-1-n]

@qfunc
def tof_CT(b0:CBool, b1:CBool, ctl0:QBit, ctl1:QBit, target:QBit):
    
    #apply NOT to target if ctl0==b0, etc
    #keeping semantics as close to original as possible
    #within (change of basis) do CCNOT
    #Note: this was not originally its own function, but this is very difficult to write inside a lambda.
    @qfunc(generative=True)
    def basischange(b0:CBool, b1:CBool, ctl0:QBit, ctl1:QBit):
        if (not b0):
            X(ctl0)
        if (not b1):
            X(ctl1)
        return

    ctls = QArray()
    bind([ctl0,ctl1],ctls)
    within_apply(
        lambda: basischange(b0,b1,ctls[0],ctls[1]),
        lambda: control(ctrl=ctls, stmt_block=lambda: X(target))
    )
    bind(ctls,[ctl0,ctl1])
    return

    



In [78]:
#type Node = CArray[CBool]

#Compute Hamming distance; rewritten slightly from Haskell for Pythonic legibility.
#Technically possible with sum(...for ...), but this is clearer.
def distance(p1: CArray[CBool], p2:CArray[CBool]):
    return len([1 for (a,b) in zip(p1,p2) if a != b])

#Increment node to next boolstring.
#Python lists aren't linked lists, and they are mutable...
#Doing this out of place, for now.
def increment(l: CArray[CBool]) -> CArray[CBool]:
    next = l.copy()
    #flip True (1) to False (0) until we hit a False(0); then flip and break.
    for i in range (len(next)-1, -1, -1):
        if not next[i]:
            next[i] = True
            break
        else:
            next[i] = False
    return next

#Translate integer n into a length-m bitstring.
def to_node(n: int, m: int) -> CArray[CBool]:
    return  [((n >> (m-i-1)) % 2 == 1) for i in range(m)]

#print(increment(to_node(127, 8)))
#[True, False, False...]

In [79]:
#step to the next node in the tree, where xs are inputs and qs are ancillas.
#Precondition 1 < |bs| = |xs| = |qs| + 1.
@qfunc(generative=True)
def stepRight(n:CArray[CBool], xs:QArray, qs:QArray) -> None:
    assert xs.len == qs.len + 1
    @qfunc(generative=True)
    def stepRight_aux(n:CArray[CBool], xs:QArray, qs:QArray) -> None:
        match distance(n, increment(n)):
            case 0:
                return
            #1 -> triangle step
            case 1:
                #q0 = qs[-1]
                #q1 = qs[-2]
                CX(qs[qs.len-2],qs[qs.len-1])
                return
            #2 -> diamond step
            case 2:
                #q0 = qs[-1]
                #q1 = qs[-2]
                #q2 = qs[-3]
                #x0 = xs[-1]
                control(ctrl=qs[qs.len-2], stmt_block=lambda: X(qs[qs.len-1]))
                tof_CT(True, False, qs[qs.len-3], xs[xs.len-1], qs[qs.len-1])
                control(ctrl=qs[qs.len-3], stmt_block=lambda: X(qs[qs.len-2]))
                return
            #else recurse
            case _:
                #note: "init" means [:-1]
                #x0 = xs[-1]
                x0 = QBit()
                #xss = xs[:-1]
                xss = QArray("xss", QBit, xs.size-1)
                bind(xs,[xss,x0])
                #q0 = qs[-1]
                q0 = QBit()
                #q1 = qs[-2]
                q1 = QBit()
                #qss = qs[:-1]
                qsss = QArray("qsss", QBit, qs.size-2)
                bind(qs,[qsss,q1,q0])
                m = n[:n.len-1]
            
                H(q0)
                TDG(q1)
                CX(x0,q1)
                T(q1)
                CX(q0,q1)
                TDG(q1)
                CX(x0,q1)
                T(q1)

                SDG(q0)
                qss = QArray("qss", QBit, qs.size-1)
                bind([qsss,q1],qss)
                stepRight_aux(m,xss,qss)
                bind(qss,[qsss,q1])

                T(q1)
                CX(x0,q1)
                T(q1)
                CX(q0,q1)
                TDG(q1)
                CX(x0,q1)
                TDG(q1)
                H(q0)

                bind([qsss,q1,q0], qs)
                bind([xss,x0],xs)
        return

    x0 = QBit()
    xss = QArray("xss", QBit, xs.size-1)
    q0 = QBit()
    qss = QArray("qss", QBit, qs.size-1)
    qs_aux = QArray("qs_aux", QBit, qs.size)
    within_apply(
        lambda : [
            bind(xs, [x0, xss]),
            bind(qs, [q0, qss]),
            bind([x0,qss],qs_aux)
        ],
        lambda : stepRight_aux(n, xss, qs_aux)
    )
    return


@qfunc(generative=True)
def walkDown(bs: CArray[CBool], xs: QArray, qs: QArray) -> None:
    @qfunc(generative=True)
    def walkDown_aux(bs: CArray[CBool], xs: QArray, qs: QArray) -> None:
        #Quantum Arrays can't be empty! We have to catch the "end" earlier.
        if(len(bs)==0):
            return
        elif(len(bs)==1):
            tof_CT(bs[0], True, xs[0], qs[0], qs[1])
        else:
            tof_CT(bs[0], True, xs[0], qs[0], qs[1])
            walkDown_aux(bs[1:], xs[1:xs.len], qs[1:qs.len])
        return

    tof_CT(bs[0], bs[1], xs[0], xs[1], qs[0])
    walkDown_aux(bs[2:], xs[2:xs.len], qs)



@qfunc(generative=True)
def walkUp(bs: CArray[CBool], xs: QArray, qs: QArray) -> None:
    invert(lambda: walkDown(bs, xs, qs))


In [80]:
#Operators = list of controlled circuits, i.e. [u(control, targets)]
#This type alias fails???
#type Operators = QCallableList[QBit, QArray]

#controlled uniformly-controlled gate: if q, then apply ops[xs](target).
@qfunc(generative=True)
def controlled_selectV(ops: QCallableList[QBit, QArray[QBit]], q: QBit, xs: QArray, target: QArray) -> None:
    m = ops.len
    n = xs.size
    start = [True] + to_node(0,n)
    end = [True] + to_node((m-1),n)

    #Operand lists don't support slicing?!
    @qfunc(generative=True)
    def applyAndStep(ops: QCallableList[QBit, QArray[QBit]], opsindex: CInt, start: CArray[CBool], end: CArray[CBool], xs: QArray, qs: QArray, target: QArray)  -> None:
        if (start == end):
            ops[opsindex](qs[qs.len-1], target)
            return
        else:
            ops[opsindex](qs[qs.len-1], target)
            stepRight(start, xs, qs)
            #applyAndStep(ops[1:ops.len], increment(start), end, xs, qs, target)
            applyAndStep(ops, opsindex+1, increment(start), end, xs, qs, target)
            return
        return

    #xs' isn't a valid Python name!
    #xs' = q:xs
    xsp = QArray("xs'", QBit, n + 1)
    qs = QArray("qs", QBit, n)
    within_apply(
        lambda: bind([q,xs], xsp),
        #with_ancilla_list n $ \qs-> do...
        #creates n ancillas in 0 and uses as qs in the function following.
        lambda: within_apply(
            lambda: allocate(n, qs),
            lambda: [
                walkDown(start, xsp, qs),
                applyAndStep(ops, 0, start, end, xsp, qs, target),
                walkUp(end, xsp, qs)
            ]
        )
    )
    return



In [81]:
from classiq.execution import ExecutionPreferences

CONTROL_LENGTH = 4
NUM_OPS = 16
NUM_SHOTS = 1000
SYSTEM_LENGTH = 16
assert (
    np.floor(np.log2(NUM_OPS)) <= CONTROL_LENGTH
), "log2(NUM_OPS) cannot be larger than the control length"

@qfunc
def main(sys: Output[QArray]) -> None:
    allocate(SYSTEM_LENGTH, sys)
    cnum = QNum("cnum", CONTROL_LENGTH, UNSIGNED, 0)
    cbit = QBit()
    #oplist = QCallableList()
    oplist = [(lambda c, ts: CX(c, ts[i])) for i in range(NUM_OPS)]
    #print(type(oplist))
    allocate_num(CONTROL_LENGTH, UNSIGNED, 0, cnum)
    allocate(cbit)
    X(cbit)
    cont = QArray("cont", QBit, cnum.size)
    bind(cnum, cont)
    controlled_selectV(oplist, cbit, cont, sys)
    bind(cont, cnum)

    #invert(lambda: assign_control(cnum))
    #X(cbit)
    #free(cbit)

In [82]:
qmod = create_model(
    main,
    execution_preferences=ExecutionPreferences(num_shots=NUM_SHOTS),
    out_file="selectVtest",
)


qprog = synthesize(qmod)

circuit_width = QuantumProgram.from_qprog(qprog).data.width
circuit_depth = QuantumProgram.from_qprog(qprog).transpiled_circuit.depth
print(f"The circuit width is {circuit_width} and the circuit_depth is {circuit_depth}")
show(qprog)

ClassiqExpansionError: 
	Index -1 is out of bounds for quantum array 'qs' (of length 2)
		at ipynb cell line 24 character 16 in function tof_CT
If you need further assistance, please reach out on our Community Slack channel at: https://short.classiq.io/join-slack or open a support ticket at: https://classiq-community.freshdesk.com/support/tickets/new